In [6]:
from mlflow.metrics import make_metric, MetricValue
import mlflow
import pandas as pd

def custom_metric_fn(predictions, targets, metrics, random):
    return MetricValue(scores=[15] * len(predictions), justifications=["hi"] * len(predictions), aggregate_results={"avg": 15})
custom_metric = make_metric(name="custom_metric", eval_fn=custom_metric_fn, greater_is_better=True)

def custom_metric_fn_2(predictions, metrics, different_params, something_random, custom_metric):
    return custom_metric.aggregate_results["avg"] * 15
custom_metric_2 = make_metric(name="custom_metric_2", eval_fn=custom_metric_fn_2, greater_is_better=True)

def custom_metric_fn_3(predictions, targets, custom_metric_2):
    return custom_metric_2.aggregate_results["custom_metric_2"] - 20
custom_metric_3 = make_metric(name="custom_metric_3", eval_fn=custom_metric_fn_3, greater_is_better=True)

def custom_metric_fn_4(predictions, targets, custom_metric, custom_metric_3):
    return None
    return custom_metric.aggregate_results["avg"] + custom_metric_3.aggregate_results["custom_metric_3"]
custom_metric_4 = make_metric(name="custom_metric_4", eval_fn=custom_metric_fn_4, greater_is_better=True)

# Create a pandas dataframe with columns 'inputs', 'predictions', 'context'
eval_df = pd.DataFrame([
    {'inputs': 'What is the capital of France?', 'predictions': 'Paris', 'targets': 'France is a country in Western Europe', 'something_random': 'blah blah', 'more_cols': 'woohoo', 'another_col': '1'},
    {'inputs': 'What is the capital of Germany?', 'predictions': 'Berlin', 'targets': 'Germany is a country in West', 'something_random': 'blah blah', 'more_cols': 'yay', 'another_col': '2'}])

eval_results = mlflow.evaluate(
    data = eval_df, # evaluation data
    evaluators="default",
    predictions="predictions", # prediction column_name from eval_df
    targets="targets",
    extra_metrics=[custom_metric_4, custom_metric_2, custom_metric, custom_metric_3],
    evaluator_config={
      "col_mapping": {"different_params": "more_cols", "random": "another_col"}
    }
)

print(eval_results.metrics)

2024/01/11 17:47:26 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.


2024/01/11 17:47:26 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/01/11 17:47:26 WARNING mlflow.models.evaluation.default_evaluator: Did not log metric 'custom_metric_4' at index 0 in the `extra_metrics` parameter because it returned None.
2024/01/11 17:47:26 WARNING mlflow.models.evaluation.default_evaluator: Did not log metric 'custom_metric_4' at index 0 in the `extra_metrics` parameter because it returned None.


{'custom_metric/avg': 15, 'custom_metric_2': 225, 'custom_metric_3': 205}


In [ ]:
import mlflow

